<a href="https://colab.research.google.com/github/sanjeeth-baliga/The-Learning-Expedition/blob/main/Semantic_Matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Consulting Co-pilot

Having spent a lot of time with the consulting industry, I came across multiple scenarios where formula based logic and rules-based workflows fell short of emulating human judgement. This code delves into how the limitations of boosting productivity in such situations can be resolved with the simple GenAI concept of semantic similarity search performed through open source libraries. I have experimented with two prominent use cases that although underappreciated, form a crucial part of every consulting engagement

*   Mapping job titles to standard department archetypes which is really a data cleanup step across projects associated with organizational rightsizing
*   Mapping value levers to their key performance indicators which is a setup step for numerous benchmarking analyses

I have also compared the results from the semantic similarity search with the ideal answer to understand the accuracy of such scripts.

In both the above use cases, the application serves as a simple co-pilot that augments human judgement to reduce the cognitive effort requirements.

**Note:** All inputs used in this program are dummy values generated by ChatGPT and only go to the extent of resembling the client data seen across consulting engagements

This experimental program will require creation of 6 csv files. The file names can be changed once their purpose is understood

**Experiment 1:**
*   Input_titles - consists of a list job titles
*   Categories - consists of department and function names against which the job titles need to be mapped
*   auto_titles - consists of the expected mapping between job titles, departments and functions

**Experiment 2:**
*   Value_lever - consists of a list of value levers
*   KPI_list - consists of a list of KPIs to be mapped against the value levers
*   vl_KPI - consists of the expected mapping between value levers and KPIs








##Perform the essential library imports

Langchain is used as the tooling framework across this application and Chroma serves as the vector datastore for performing the semantic similarity search. The appropriate modules are installed here for the same purpose

In [1]:
!pip install --upgrade langchain -q
!pip install --upgrade chromadb -q
!pip install InstructorEmbedding -q
!pip install sentence_transformers --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain.vectorstores.chroma import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import pandas as pd

Storing data into a vectorstore database requires translation of text into embedding vectors that can mapped to a multi-dimensional vectorspace

In [3]:
#Define the embedding object to map the text data to vectorspace for storage into a vectorstore
data_embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

##Define a mapping function

This function implements the semantic similarity search process for all items (e.g. job titles) that need to be categorized. This search is performed against the category entities (e.g. departments, functions, etc.) which are stored in a vectorstore

In [4]:
def map_category(category_file,item_data,pref_count):
  mapping_categories = pd.read_csv(category_file)

  #Read the item data that needs to be mapped into suitable categories
  inputs = item_data

  #Use the langchain CSVLoader to read the category data from a CSV file for subsequent storage as embedding vectors into a vectorspace
  loader = CSVLoader(file_path=category_file)
  stored_chunks = loader.load()

  #The Chroma database is used as the vectorstore to organize the categorical entities
  memory = Chroma.from_documents(documents=stored_chunks,embedding=data_embedding)

  #A dictionary is initialized here to store the matched results while keying them to the right category entity names.
  #A list of dictionaries are created here to store mappings at multiple levels of closeness of match (e.g. dictionary at index 0 stores the closest match,
  #dictionary at index 1 stores the next closest match and so on)
  pref_dict = [{key:list() for key in ['input']+mapping_categories.columns.to_list()} for cnt in range(pref_count)]

  for input in inputs[inputs.columns[0]].to_list():
    #Perform a similarity search on the vectorstore database corresponding to each element in the item data (e.g., job titles)
    #The parameter k refers to the number of closest matches that should be generated
    matches= memory.similarity_search(input,k=pref_count)
    for i in range(pref_count):
      #Store the item data in the 'input' field of each dictionary for ease of validation
      pref_dict[i]['input'].append(input)
    for key in mapping_categories.columns.to_list():
      for i in range(pref_count):
        #Store the match generated corresponding to each entity category keyed by the category name into dictionaries organized serially by closeness of match
        pref_dict[i][key].append(mapping_categories.loc[matches[i].metadata['row'],key])

  #Delete the vectorstore entries to prevent interference with the next function call
  memory.delete_collection()
  #Create a pandas dataframe out of each matching dictionary and return the list of dataframes
  return [pd.DataFrame(mapping) for mapping in pref_dict]

##Experiment 1
Involves mapping a job title to the right department and function from a laundry list of standard entities. The semantic similarity search algorithm returns a list of suggested mappings that can be then subjected to manual judgement instead of performing the entire process from scratch

In [6]:
#Load the list of all job titles
job_titles = pd.read_csv('/content/Input_titles.csv')
#Performing a random shuffling of job titles for the purpose of validation
job_titles = job_titles.sample(frac=1)
job_titles.head(10)

,Job Title
42,Senior Sales Engineer
34,Facilities Manager
26,Talent Acquisition Specialist
12,Logistics Manager
21,Regulatory Affairs Specialist
8,Human Resources Generalist
1,Manufacturing Supervisor
30,Business Development Representative
37,Senior Regulatory Compliance Officer
33,IT Project Manager


In [7]:
#Load the departments and functions to which the job titles need to be mapped (this is just for visualization purposes)
dept_func = pd.read_csv('/content/Categories.csv')
dept_func.head(10)

,Department,Function
0,Engineering,Software Development
1,Manufacturing,Production Supervision
2,Finance,Financial Analysis
3,Product Management,Product Development
4,Customer Service,Customer Support
5,Supply Chain,Supply Chain Analysis
6,Marketing,Marketing Coordination
7,Quality Assurance,Quality Control
8,Human Resources,HR Generalist
9,Information Technology,Systems Administration


In [8]:
#Invoke the function for performing matches. Assume only the closest match is required for now
matched_result = map_category('/content/Categories.csv',job_titles,1)
#A quick glance shows that semantic similarity has done a fairly good job at generating matches
matched_result[0].head(10)

,input,Department,Function
0,Senior Sales Engineer,Sales,Sales Engineering
1,Facilities Manager,Facilities,Facilities Management
2,Talent Acquisition Specialist,Human Resources,Talent Acquisition
3,Logistics Manager,Logistics,Logistics Management
4,Regulatory Affairs Specialist,Legal/Compliance,Regulatory Affairs
5,Human Resources Generalist,Human Resources,HR Generalist
6,Manufacturing Supervisor,Manufacturing,Production Supervision
7,Business Development Representative,Business Development,Business Development
8,Senior Regulatory Compliance Officer,Legal/Compliance,Regulatory Affairs
9,IT Project Manager,Information Technology,Project Management


Validate the accuracy of mapping by comparing the generated result against the expected mappings

In [9]:
mapping_out = matched_result[0][['input','Department']]
#Read the expected results into a dataframe
mapping_actual = pd.read_csv('/content/auto_titles.csv',header=0,names=['input','Actual department','Function'])[['input','Actual department']]

#Merge the mapping result dataframe with the expected result dataframe to ensure alignment by job title in a single dataframe
validation_frame = pd.merge(mapping_out,mapping_actual,on='input')

#Evaluate whether the mapping result of every row is same as the expected mapping of the same row
matches = validation_frame.apply(lambda x: 'Match' if x['Department']==x['Actual department'] else 'No_Match',axis=1)

#Calculate the accuracy of mapping based on the evaluation
outcomes = matches.value_counts()

if 'No_Match' in outcomes.index:
  accuracy = outcomes['Match']/(outcomes['Match']+outcomes['No_Match'])
else:
  accuracy = 1
accuracy

1

#Experiment 2

Involves mapping a value lever to an appropriate key performance indicator (KPI) that would help measure it. This will create a set of suggested KPI mappings against each value lever and will serve as a co-pilot to improve work productivity

In [11]:
#Load the list of all value levers
value_levers = pd.read_csv('/content/Value_lever.csv')
#Randomly shuffle the value levers for the purpose of validation
value_levers = value_levers.sample(frac=1)
value_levers.head(10)

,Value Levers and Detailed Explanation
4,Operational Efficiency and Cost Control: Ident...
5,Sustainable and Ethical Practices: Integrating...
17,Regulatory Compliance and Quality Standards: A...
22,Packaging Innovation and Sustainability: Intro...
34,Water and Waste Management: Implementing effic...
19,Corporate Social Responsibility (CSR) Initiati...
12,Costumer Feedback and Satisfaction: Actively s...
9,Digital Transformation and E-commerce Adoption...
20,Employee Training and Development Programs: In...
26,Price Optimization and Competitive Analysis: A...


In [12]:
#Load the list of all KPIs (this is just for initial visualization purposes)
KPI_list = pd.read_csv('/content/KPI_list.csv')
KPI_list.head(10)

,Key Performance Indicator (KPI)
0,Number of new products launched
1,Inventory turnover ratio
2,Brand equity index
3,Customer satisfaction scores
4,Operating profit margin
5,Sustainability index rating
6,Market penetration rate
7,Time-to-insight from data analysis
8,Employee satisfaction and engagement
9,Online sales growth


In [13]:
#Invoke the function for performing matches. Assume that the top 3 closest matches are extracted
Lever_KPI_matches = map_category('/content/KPI_list.csv',value_levers,3)

#Merge the results across the 3 mappings to obtain a single frame where the mappings are aligned by value lever
for i,option in enumerate(Lever_KPI_matches):
  #Ensure that the columns have unqiue column names
  option.rename(columns={option.columns[1]:f"choice_{i}"},inplace=True)
mapping_options = pd.concat(Lever_KPI_matches,axis='columns')
mapping_options = mapping_options.iloc[0:,[0,1,3,5]]
mapping_options.head(5)

,input,choice_0,choice_1,choice_2
0,Operational Efficiency and Cost Control: Ident...,Operating profit margin,Business continuity plan effectiveness,Supplier satisfaction and reliability
1,Sustainable and Ethical Practices: Integrating...,Sustainability index rating,Adoption rate of sustainable packaging solutions,Percentage of recyclable packaging used
2,Regulatory Compliance and Quality Standards: A...,Compliance with quality standards,Number of product safety incidents,Supplier satisfaction and reliability
3,Packaging Innovation and Sustainability: Intro...,Adoption rate of sustainable packaging solutions,Percentage of recyclable packaging used,Sustainability index rating
4,Water and Waste Management: Implementing effic...,Reduction in water consumption and waste gener...,Reduction in energy consumption and environmen...,Sustainability index rating


In [14]:
#Read the data representing the correct expected mappings into a dataframe
correct_mapping = pd.read_csv('/content/vl_KPI.csv',header=0,names=['input','actual'])

#Bring the expected mapping into a new column named 'actual' in the mappings dataframe
mapping_validate = pd.merge(mapping_options,correct_mapping,on='input')
mapping_validate.head(7)

,input,choice_0,choice_1,choice_2,actual
0,Operational Efficiency and Cost Control: Ident...,Operating profit margin,Business continuity plan effectiveness,Supplier satisfaction and reliability,Operating profit margin
1,Sustainable and Ethical Practices: Integrating...,Sustainability index rating,Adoption rate of sustainable packaging solutions,Percentage of recyclable packaging used,Sustainability index rating
2,Regulatory Compliance and Quality Standards: A...,Compliance with quality standards,Number of product safety incidents,Supplier satisfaction and reliability,Compliance with quality standards
3,Packaging Innovation and Sustainability: Intro...,Adoption rate of sustainable packaging solutions,Percentage of recyclable packaging used,Sustainability index rating,Adoption rate of sustainable packaging solutions
4,Water and Waste Management: Implementing effic...,Reduction in water consumption and waste gener...,Reduction in energy consumption and environmen...,Sustainability index rating,Reduction in water consumption and waste gener...
5,Corporate Social Responsibility (CSR) Initiati...,Impact metrics from CSR initiatives,Sustainability index rating,Business continuity plan effectiveness,Impact metrics from CSR initiatives
6,Costumer Feedback and Satisfaction: Actively s...,Customer feedback scores,Customer satisfaction scores,Employee satisfaction and engagement,Customer feedback scores


In [15]:
#Evaluate whether the expected mapping is found among the potential mappings across other columns returned by semantic similarity search
matches_in_top_choice = mapping_validate.apply(lambda x: 'Match' if x['actual'] in x[mapping_validate.columns[1:-1]].tolist() else 'No_Match',axis=1)
outcomes = matches_in_top_choice.value_counts()
#Determine the accuracy based on the evaluation performed above
accuracy = outcomes['Match']/(outcomes['Match']+outcomes['No_Match'])

In [16]:
accuracy

0.8108108108108109